In [ ]:
import pandas as pd
import numpy as np


test_x = pd.read_csv('test_set_x.csv', header=0)

In [ ]:
test_x.shape

In [ ]:
test_x.columns

In [ ]:
#test_x.drop(['MACHINE_ID', 'MACHINE_DATA'], inplace=True, axis=1)
test_x['STAGE'] = test_x['STAGE'].map(lambda x: 1 if x == 'A' else 0)
test_x['TIMESTAMP'] = test_x['TIMESTAMP'] - test_x['TIMESTAMP'].min()

test_x.info()

In [ ]:
from scipy.integrate import trapz # 用于计算积分
 

def cal_integral(x,y):
    integrals = []
    for i in range(len(y)): # 计算梯形的面积，由于是累加，所以是切片"i+1"
        integrals.append(trapz(y[:i + 1], x[:i + 1]))
    
    return integrals

In [ ]:
inte_features = ['USAGE_OF_BACKING_FILM', 'USAGE_OF_DRESSER',
       'USAGE_OF_POLISHING_TABLE', 'USAGE_OF_DRESSER_TABLE',
       'PRESSURIZED_CHAMBER_PRESSURE', 'MAIN_OUTER_AIR_BAG_PRESSURE',
       'CENTER_AIR_BAG_PRESSURE', 'RETAINER_RING_PRESSURE',
       'RIPPLE_AIR_BAG_PRESSURE', 'USAGE_OF_MEMBRANE',
       'USAGE_OF_PRESSURIZED_SHEET', 'SLURRY_FLOW_LINE_A',
       'SLURRY_FLOW_LINE_B', 'SLURRY_FLOW_LINE_C', 'WAFER_ROTATION',
       'STAGE_ROTATION', 'HEAD_ROTATION', 'DRESSING_WATER_STATUS',
       'EDGE_AIR_BAG_PRESSURE']


def grp_proc(s):
    row = { 'WAFER_ID': [s['WAFER_ID'].unique()[0]],
            'PATH_CHAMBERS': [len(s['CHAMBER'].unique())],
            'PATH_STAGES': [len(s['STAGE'].unique())],
            'START_TIME': [s['TIMESTAMP'].min()],
            'RUN_TIME': [s['TIMESTAMP'].max() - s['TIMESTAMP'].min()],
            'CHAMBER': [np.sum(list(s['CHAMBER'].unique()))],
        }

    for i in inte_features:
        row[i] = trapz(s[i], s['TIMESTAMP'])/row['RUN_TIME'][0]

    return pd.DataFrame.from_dict(row)



#test = grp_proc(df_wafer)
#print(test)

X_grp0 = test_x.groupby(['WAFER_ID'], as_index=False).apply(grp_proc)
X_grp0.shape
print(X_grp0)

In [ ]:
X_grp0.to_csv('test_x_compresssd_1.csv', index=False)

In [ ]:

wafer = test_x['WAFER_ID'].unique()[0:1]

df_wafer = test_x[test_x['WAFER_ID'].isin(wafer)]
print(df_wafer.shape)

df_wafer.drop_duplicates(subset=['TIMESTAMP'], inplace=True)
print(df_wafer.shape)


In [ ]:
from scipy import interpolate

def grp_cpr(s):
    cols = list(set(s.columns)-set(['TIMESTAMP']))

    X = df_wafer['TIMESTAMP']
    X_view = np.linspace(X.iloc[0], X.iloc[-1], 50)
    s2 = pd.DataFrame(X_view, columns=['TIMESTAMP'])
    for i in cols:
        tck = interpolate.make_interp_spline(x=X, y=s[i], k=1)
        piecewise_polynomial = interpolate.PPoly.from_spline(tck, extrapolate=None)
        s2[i] = piecewise_polynomial(X_view)

    return s2


test = grp_cpr(df_wafer)
test.shape
print(test)